In [1]:
!pip install tf-nightly
import tensorflow as tf
import numpy as np
import os

     |████████████████████████████████| 391.7MB 41kB/s 
     |████████████████████████████████| 10.2MB 12.0MB/s 
     |████████████████████████████████| 471kB 39.6MB/s 


In [2]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True, cache_subdir='/content',)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 7s 0us/step


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

Found 2939 images belonging to 5 classes.


In [4]:
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 731 images belonging to 5 classes.


In [5]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 5))

**meaning that in the first batch of train_generator, 64 images of dimensions 224 x 224 x 3 were created, along with 64 labels of 5 one-hot encoding formatting.**

In [7]:
# checking assigned encoding values - notice the alphabetical order
print(train_generator.class_indices)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


**Save these labels for future usage to deploy the model in the Flutter application**

In [12]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [13]:
labels

'daisy\ndandelion\nroses\nsunflowers\ntulips'

## Creating a base model and freezing all the layers except the last one

In [14]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


**Now, we freeze the base model so that the weights in the MobileNetV2 model remain unaffected by future training, as follows:**

In [16]:
base_model.trainable = False

**Next we will create an extended CNN, and extend the base model to add another layer after the base model layers, like this:**

In [17]:
model = tf.keras.Sequential([
      base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 2,626,821
Trainable params: 368,837
Non-trainable params: 2,257,984
_________________________________________________________________


**Compiling the model to perform training on it**

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

##### We set the loss as categorical cross-entropy and the model evaluation metric as the accuracy of prediction. Softmax has been found to perform best with categorical cross-entropy as the loss function, hence the choice. 

**Train and save the model**

In [22]:
epochs=10
model.fit(train_generator,
         epochs=epochs,
         validation_data=val_generator)


Epoch 1/10
46/46 [==============================] - 147s 3s/step - loss: 1.3411 - accuracy: 0.5763 - val_loss: 0.5538 - val_accuracy: 0.8372
Epoch 2/10
46/46 [==============================] - 143s 3s/step - loss: 0.3582 - accuracy: 0.8705 - val_loss: 0.5127 - val_accuracy: 0.8290
Epoch 3/10
46/46 [==============================] - 141s 3s/step - loss: 0.1837 - accuracy: 0.9391 - val_loss: 0.4477 - val_accuracy: 0.8605
Epoch 4/10
46/46 [==============================] - 141s 3s/step - loss: 0.1184 - accuracy: 0.9615 - val_loss: 0.4718 - val_accuracy: 0.8632
Epoch 5/10
46/46 [==============================] - 139s 3s/step - loss: 0.0621 - accuracy: 0.9822 - val_loss: 0.4984 - val_accuracy: 0.8605
Epoch 6/10
46/46 [==============================] - 140s 3s/step - loss: 0.0402 - accuracy: 0.9929 - val_loss: 0.5336 - val_accuracy: 0.8714
Epoch 7/10
46/46 [==============================] - 141s 3s/step - loss: 0.0352 - accuracy: 0.9920 - val_loss: 0.5504 - val_accuracy: 0.8618
Epoch 8/10
46

###### The model is trained for 10 epochs, which means every sample is thrown at the neural network at least 10 times. Notice the use of train_generator and val_generator in this function. The training takes quite some time on this, even with 12GB+ of RAM and TPU acceleration available (which would be an overkill on any personal, mid-end device). You'll be able to observe the training logs below the cell that runs the preceding code.  

**We can then save the model, after which we can proceed to convert the saved model file,**

In [23]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2292: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:1377: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


INFO:tensorflow:Assets written to: assets


###### Convert and download the model file to TensorFlow Lite. We can now convert the saved model file using the following code. This saves the model as a model.tflite

In [24]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

#### We now need to download this file, for embedding it into the Flutter application that we build. We can do so using the following code:

In [25]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>